In [2]:
# %load LuiseUtil.py
import os
import shutil
import math
from ete3 import Tree
from Bio import Phylo
import matplotlib
import matplotlib.pyplot as plt

tree_dir = '../data/trees/'
alignment_dir = '../data/language_alignments/'
sitelh_dir = '../data/siteLH/'
drawings_dir = '../output/drawings/'
weight_calibration_dir = '../data/weight_calibration/'
site_congruence_dir = '../data/site_congruence/'

tree_space_name = 'space.trees'
geo_tree_name = "geo_basic.tree"
cognate_tree_name = "cognate_ie_compatible.tree"
cognate_ml_tree_name = "cognate_ml.tree"

morpho_alignment_name = "morpho.phy"



def read_trees_from_ete(tree_set_names):
    trees = []
    for tree_set in tree_set_names:
        l_file = open(tree_dir + tree_set, 'r')
        lines = l_file.readlines()
        for line in lines:
            trees.append(Tree(line))
    print(str(len(trees)) + " trees read")
    return trees

def read_tree_space_ete():
    return read_trees_from_ete([tree_space_name])


def eliminate_topological_duplicates_ete(tree_set_name):
    unique_list = []
    tree_set_ete = read_trees_from_ete([tree_set_name])
    for t1 in tree_set_ete:
        unique = True
        for t2 in unique_list:
            rf = rf_distance_ete(t1, t2)
            if rf == 0:
                unique = False
                break
        if unique:
            unique_list.append(t1)
    file_name = tree_dir + rm_end(tree_set_name) + '_unique.trees'
    with open(file_name, 'w+') as tree_file:
        for tree in unique_list:
            tree_file.write(tree.write()+"\n")



def create_tree_space_from(tree_set_names):
    tree_space_ete = read_trees_from_ete(tree_set_names)
    file_name = tree_dir + tree_space_name
    with open(file_name, 'w+') as tree_file:
        for tree in tree_space_ete:
            tree_file.write(tree.write()+"\n")
    print(str(len(tree_space_ete)) + " trees written to " + file_name)

def read_geo_tree_ete():
    return Tree(tree_dir + geo_tree_name)

def read_cognate_tree_ete():
    return Tree(tree_dir + cognate_tree_name)

def rf_distance_ete(t1, t2):
    rf, max_rf, common_leaves, parts_t1, parts_t2,discard_t1, discart_t2 = t1.robinson_foulds(t2, unrooted_trees = True)
    if max_rf == 0:
        return 0
    return rf/max_rf

def rf_distances_ete(ref_tree, tree_space):
    distances = []
    for tree in tree_space:
        distances.append(rf_distance_ete(ref_tree, tree))
    return distances


def calculate_rf_distances_raxml(ref_tree_name, tree_set_names):
    shutil.rmtree("temp/", ignore_errors=True)
    os.mkdir("temp/")
    dir_string = tree_dir + ref_tree_name
    for tree_set in tree_set_names:
        dir_string = dir_string + tree_dir + tree_set
    os.system("cat " + dir_string + " > temp/all.trees")
    os.system("./../raxml-ng/build/bin/raxml-ng --rfdist --tree temp/all.trees --prefix temp/foo > temp/bar.txt")
    l_file = open('temp/foo.raxml.rfDistances', 'r')
    lines = l_file.readlines()
    i = 0
    line = lines[i].split("\t")
    distances = []
    while(line[0] == '0'):
        distances.append(float(line[3]))
        i+=1
        line = lines[i].split("\t")
    shutil.rmtree("temp/", ignore_errors=True)
    return distances


def evaluate_lh_raxml(tree_name, alignment_name, optimize = True):
    optimize_string = ""
    if not optimize:
        optimize_string = " --opt-branches off "
    os.system('./../raxml-ng/build/bin/raxml-ng --evaluate --msa ' + alignment_dir + alignment_name +
            ' --threads 2 --model BIN+G --tree '  + tree_dir + tree_name +  ' --prefix foo --nofiles' +
              optimize_string + '> out.txt')
    l_file = open('out.txt', 'r')
    lines = l_file.readlines()
    lh = 0
    for line in lines:
        if(line.startswith('Final LogLikelihood:')):
            lh = float(line.split(" ")[2].strip())
    os.remove("out.txt")
    return lh




def calculate_site_lh_raxml_ete(tree_ete, alignment_name, optimize= True):
    shutil.rmtree("temp/", ignore_errors=True)
    os.mkdir("temp/")
    tree_ete.write(outfile="temp/foo.tree")
    optimize_string = ""
    if not optimize:
        optimize_string = " --opt-branches off "
    os.system('./../raxml-ng/build/bin/raxml-ng --sitelh --msa ' + alignment_dir + alignment_name +
            ' --threads 2 --model BIN+G --tree temp/foo.tree --prefix temp/foo ' +
              optimize_string + '> temp/bar.txt')
    l_file = open('temp/foo.raxml.log', 'r')
    lines = l_file.readlines()
    lh = 0
    for line in lines:
        if(line.startswith('Final LogLikelihood:')):
            lh = float(line.split(" ")[2].strip())
    with open('temp/foo.raxml.siteLH' , 'r') as file:
        data = (file.read().replace('\n', '')).split(" ")
    siteLH = [float(data[i]) for i in range(5, len(data))]
    shutil.rmtree("temp/", ignore_errors=True)
    return [lh, siteLH]




def print_tree_with_phylo(tree_name, save = False):
    tree = Phylo.read(tree_dir + tree_name, "newick")
    tree.ladderize()
    fig = plt.figure(figsize=(10, 10), dpi=100)
    axes = fig.add_subplot(1, 1, 1)
    axes.set_title(tree_name)
    Phylo.draw(tree, axes=axes, do_show=False)
    if save:
        plt.savefig(drawings_dir + tree_name + '.png', dpi=fig.dpi)


def fix_beast_output(tree_set_name):
    beast_file = open(tree_dir + tree_set_name, 'r')
    lines = beast_file.readlines()
    i = 0
    while not lines[i].startswith("\tTranslate"):
        i = i+1
    translate = []
    while not lines[i].startswith(";"):
        if lines[i].endswith(",\n"):
            translate.append(lines[i].split(" ")[-1][:-2])
        else:
            translate.append(lines[i].split(" ")[-1][:-1])
        i=i+1
    i = i+1
    with open(tree_dir + rm_end(tree_set_name) + "_fixed.trees" , 'w+') as fixed_file:
        for j in range(i, len(lines)-1):
            tree = Tree(lines[j].split(" ")[-1])
            for leaf in tree.iter_leaves():
                leaf.name = translate[int(leaf.name)]
            fixed_file.write(tree.write() + "\n")

def rm_end(file_name):
    return '.'.join(file_name.split('.') [:-1])


def site_lh_file_name(tree_name, alignment_name, optimize):
    optimize_string = "_opt-branches="
    if optimize:
        optimize_string = optimize_string + "on"
    else:
        optimize_string = optimize_string + "off"
    return sitelh_dir + rm_end(alignment_name) + '_' + rm_end(tree_name)  + optimize_string + '.raxml.siteLH'

def weight_calibration_file_name(tree_name, alignment_name):
    return weight_calibration_dir + rm_end(alignment_name) + '_' + rm_end(tree_name)  + '.raxml.weightCalibration'

def site_congruence_file_name(tree_name, alignment_name):
    return site_congruence_dir + rm_end(alignment_name) + '_' + rm_end(tree_name)  + '.raxml.siteCongruence'

def optimized_tree_file_name(tree_name, alignment_name):
    return tree_dir + rm_end(tree_name) + '_optimized_' + rm_end(alignment_name)  + '.tree'


def read_site_lh(tree_name, alignment_name, optimize):
    with open(site_lh_file_name(tree_name, alignment_name, optimize) , 'r') as file:
        data = (file.read().replace('\n', '')).split(" ")
    return [float(data[i]) for i in range(5, len(data))]

def read_weight_calibration(tree_name, alignment_name):
    with open(weight_calibration_file_name(tree_name, alignment_name) , 'r') as file:
        data = file.read().split(" ")
    return [int(data[i]) for i in range(len(data) - 1)]

def read_site_congruence(tree_name, alignment_name):
    with open(site_congruence_file_name(tree_name, alignment_name) , 'r') as file:
        data = file.read().split("\n")
    return [float(data[i].split(" ")[1]) for i in range(len(data) - 1)]

def read_optimized_tree(tree_name, alignment_name):
    return Tree(optimized_tree_file_name(tree_name, alignment_name))

def calculate_site_lh_raxml(tree_name, alignment_name, optimize= True):
    shutil.rmtree("temp/", ignore_errors=True)
    os.mkdir("temp/")
    optimize_string = ""
    if not optimize:
        optimize_string = " --opt-branches off "
    os.system('./../raxml-ng/build/bin/raxml-ng --sitelh --msa ' + alignment_dir + alignment_name +
            ' --threads 2 --model BIN+G --tree '  + tree_dir + tree_name +  ' --prefix temp/foo ' +
              optimize_string + '> temp/bar.txt')
    l_file = open('temp/foo.raxml.log', 'r')
    lines = l_file.readlines()
    lh = 0
    for line in lines:
        if(line.startswith('Final LogLikelihood:')):
            lh = float(line.split(" ")[2].strip())
    os.system("cat temp/foo.raxml.siteLH > " + site_lh_file_name(tree_name, alignment_name, optimize))
    with open('temp/foo.raxml.siteLH' , 'r') as file:
        data = (file.read().replace('\n', '')).split(" ")
    siteLH = [float(data[i]) for i in range(5, len(data))]
    shutil.rmtree("temp/", ignore_errors=True)
    return [lh, siteLH]

def calculate_weight_calibration_raxml(tree_name, alignment_name):
    os.system('./../standard-RAxML-master/raxmlHPC -f u -p 12345 -t ' + tree_dir + tree_name +
              ' -m BINGAMMA -s ' + alignment_dir + alignment_name +
              ' -n calibration > bar.txt')
    os.system('cat RAxML_weights.calibration > '
              + weight_calibration_file_name(tree_name, alignment_name))
    os.remove('bar.txt')
    os.remove('RAxML_weights.calibration')
    os.remove('RAxML_info.calibration')

def calculate_site_congruence_raxml(tree_name, alignment_name):
    os.system('./../standard-RAxML-master/raxmlHPC -f S -t ' + tree_dir + tree_name +
              ' -m BINGAMMA -s ' + alignment_dir + alignment_name +
              ' -n congruence > bar.txt')
    os.system('cat RAxML_SiteSpecificPlacementBias.congruence > '
              + site_congruence_file_name(tree_name, alignment_name))
    os.remove('bar.txt')
    os.remove('RAxML_SiteSpecificPlacementBias.congruence')
    os.remove('RAxML_info.congruence')

def calculate_optimized_tree_raxml(tree_name, alignment_name):
    shutil.rmtree("temp/", ignore_errors=True)
    os.mkdir("temp/")
    os.system('./../raxml-ng/build/bin/raxml-ng --evaluate --msa ' + alignment_dir + alignment_name +
            ' --threads 2 --model BIN+G --tree '  + tree_dir + tree_name +  ' --prefix temp/foo ' + '> out.txt')
    os.system('cat temp/foo.raxml.bestTree > ' + tree_dir + rm_end(tree_name) + "_optimized_" + rm_end(alignment_name) + '.tree')
    shutil.rmtree("temp/", ignore_errors=True)


def get_site_lh(tree_name, alignment_name, optimize):
    if not os.path.isfile(site_lh_file_name(tree_name, alignment_name, optimize)):
        calculate_site_lh_raxml(tree_name, alignment_name, optimize)
    return read_site_lh(tree_name, alignment_name, optimize)


def get_weight_calibration(tree_name, alignment_name):
    if not os.path.isfile(weight_calibration_file_name(tree_name, alignment_name)):
        calculate_weight_calibration_raxml(tree_name, alignment_name)
    return read_weight_calibration(tree_name, alignment_name)

def get_site_congruence(tree_name, alignment_name):
    if not os.path.isfile(site_congruence_file_name(tree_name, alignment_name)):
        calculate_site_congruence_raxml(tree_name, alignment_name)
    return read_site_congruence(tree_name, alignment_name)

def get_optimized_tree(tree_name, alignment_name):
    if not os.path.isfile(optimized_tree_file_name(tree_name, alignment_name)):
        calculate_optimized_tree_raxml(tree_name, alignment_name)
    return read_optimized_tree(tree_name, alignment_name)

def get_double_optimized_tree(tree_name, alignment_name):
    if not os.path.isfile(optimized_tree_file_name(tree_name, alignment_name)):
        calculate_optimized_tree_raxml(tree_name, alignment_name)
    optimized_tree_name = optimized_tree_file_name(tree_name, alignment_name).split('/')[-1]
    if not os.path.isfile(optimized_tree_file_name(optimized_tree_name, alignment_name)):
        calculate_optimized_tree_raxml(optimized_tree_name, alignment_name)
    return read_optimized_tree(optimized_tree_name, alignment_name)



def average_branch_length(tree_space):
    avg = 0
    cnt = 0
    for tree in tree_space:
        for node in tree.traverse():
            avg = avg + node.dist
            cnt = cnt + 1
    avg = avg / cnt
    return avg



def interval_branch_length(tree_space):
    lower = 1
    upper = 0
    for tree in tree_space:
        for node in tree.traverse():
            lower = min(lower, node.dist)
            upper = max(upper, node.dist)
    return (lower, upper)



def variance_branch_length(tree_space):
    avg = average_branch_length(tree_space)
    var = 0
    cnt = 0
    for tree in tree_space:
        for node in tree.traverse():
            diff = node.dist - avg
            var = var + (diff * diff)
            cnt = cnt + 1
    return var / cnt







In [3]:
def find_subtree(node):
    if not node.name == '':
        return set([node.name])
    else:
        subtree = set()
        for desc in node.iter_descendants("postorder"):
            subtree = subtree.union(find_subtree(desc))
        return subtree

def nodes_aligned(t1, t2):
    nodes1 = [node for node in t1.traverse()]
    nodes2 = [node for node in t2.traverse()]
    if not len(nodes1) == len(nodes2):
        return False
    for i in range(len(nodes1)):
        if not find_subtree(nodes1[i]) == find_subtree(nodes2[i]):
            return False
    return True



def analyse_branch_length_optimization(tree_name, alignment_name):
    tree = Tree(tree_dir + tree_name)
    opt_tree = get_optimized_tree(tree_name, alignment_name)
    compare_trees(tree, opt_tree)
    
def compare_trees(tree, opt_tree, verbose = False):
    if not nodes_aligned(tree, opt_tree):
        print("Trees not aligned")
        return
    opt_nodes = [node for node in opt_tree.traverse()]
    i = 0
    avg_rel = 0
    avg_abs = 0
    for node in tree.traverse():
        brlen = node.dist
        opt_brlen = opt_nodes[i].dist
        diff = opt_brlen - brlen
        rel = 0
        if brlen == 0:
            if opt_brlen == 0:
                rel = 0
            else:
                rel = 1
        else:
            rel = abs(diff) / brlen
        if verbose:
            print(str(brlen) + " --> " + str(opt_brlen) + " (" + str(round(rel, 3)) + ")")
        i = i + 1
        avg_rel = avg_rel + rel
        avg_abs = avg_abs + abs(diff)
    avg_rel = avg_rel / i
    avg_abs = avg_abs / i
    print("Average absolute change: " + str(avg_abs))
    print("Average relative change: " + str(avg_rel))

def create_annotated_tree(tree_name):
    ref_tree_name = rm_end(tree_name) + '_optimized.tree'
    tree = Tree(tree_dir + tree_name)
    ref_tree = Tree(tree_dir + ref_tree_name)
    if not nodes_aligned(tree, ref_tree):
        return
    ref_nodes = [node for node in ref_tree.traverse()]
    i = 0
    for node in tree.traverse():
        brlen = node.dist
        ref_brlen = ref_nodes[i].dist
        diff = ref_brlen - brlen
        c = ""
        if (not brlen == 0)  and abs(diff) / brlen < 0.1:
            c = "yellow"
        else: 
            if diff > 0:
                c = "green"
            else:
                c = "red"
        node.add_features(color=c, optimized_brlen=ref_brlen)
        i = i + 1
    with open(tree_dir + rm_end(tree_name) + '_annotated.tree',  'w+') as tree_file:
        tree_file.write(tree.write(features=["color", "optimized_brlen"]))
        
        

In [4]:
def analyze_double_optimization(tree_name, alignment_name):
    tree = Tree(tree_dir + tree_name)
    optimized_tree = get_optimized_tree(tree_name, alignment_name)
    double_optimized_tree = get_double_optimized_tree(tree_name, alignment_name)
    
    print("Average Branch Length")
    print("Original: " + str(average_branch_length([tree])))
    print("Optimized: " + str(average_branch_length([optimized_tree])))
    print("Double optimized: " + str(average_branch_length([double_optimized_tree])))
    
    print("First Optimizaion")
    compare_trees(tree, optimized_tree, False)
    
    print("Second Optimizaion")
    compare_trees(optimized_tree, double_optimized_tree, False)
    

In [22]:
geo_tree = read_geo_tree_ete()
optimized_tree = get_optimized_tree(geo_tree_name, morpho_alignment_name)
double_optimized_tree = get_optimized_tree('geo_basic_optimized_morpho.tree', morpho_alignment_name)
#print(evaluate_lh_raxml(geo_tree_name, morpho_alignment_name, True))
#print(evaluate_lh_raxml('geo_basic_optimized_morpho.tree', morpho_alignment_name, False))
#print(evaluate_lh_raxml('geo_basic_optimized_morpho.tree', morpho_alignment_name, True))
#analyse_branch_length_optimization("geo_basic_scaled_interval=[0,0.3].tree", morpho_alignment_name)
#analyse_branch_length_optimization(geo_tree_name, morpho_alignment_name)
#compare_trees(get_optimized_tree("geo_basic_scaled_interval=[0,0.3].tree", morpho_alignment_name), get_optimized_tree(geo_tree_name, morpho_alignment_name), True)

#analyze_double_optimization(geo_tree_name, morpho_alignment_name)



si_tree_name = "geo_basic_scaled_interval=[0,0.3].tree"
sa_tree_name = "geo_basic_scaled_avg=0.03.tree"

#analyze_double_optimization(si_tree_name, morpho_alignment_name)
#analyze_double_optimization(sa_tree_name, morpho_alignment_name)

si_optimized_tree = Tree(tree_dir + "geo_basic_optimized_morpho_scaled_interval=[0,0.3].tree")
sa_optimized_tree = Tree(tree_dir + "geo_basic_optimized_morpho_scaled_avg=0.03.tree")

#compare_trees(si_optimized_tree, get_optimized_tree(si_tree_name, morpho_alignment_name), True)
#print(" ")
#compare_trees(sa_optimized_tree, get_optimized_tree(sa_tree_name, morpho_alignment_name), True)
   

In [21]:
geo_tree = Tree(tree_dir + 'geo_science.tree')
optimized_tree = get_optimized_tree('geo_science.tree', morpho_alignment_name)
double_optimized_tree = get_optimized_tree('geo_science_optimized_morpho.tree', morpho_alignment_name)
#print(evaluate_lh_raxml('geo_science.tree', morpho_alignment_name, True))
#print(evaluate_lh_raxml('geo_science_optimized_morpho.tree', morpho_alignment_name, False))
#print(evaluate_lh_raxml('geo_science_optimized_morpho.tree', morpho_alignment_name, True))
#analyse_branch_length_optimization("geo_science_scaled_interval=[0,0.3].tree", morpho_alignment_name)
#analyse_branch_length_optimization('geo_science.tree', morpho_alignment_name)
#compare_trees(get_optimized_tree("geo_science_scaled_interval=[0,0.3].tree", morpho_alignment_name), get_optimized_tree('geo_science.tree', morpho_alignment_name), True)

#analyze_double_optimization('geo_science.tree', morpho_alignment_name)



si_tree_name = "geo_science_scaled_interval=[0,0.3].tree"
sa_tree_name = "geo_science_scaled_avg=0.03.tree"

#analyze_double_optimization(si_tree_name, morpho_alignment_name)
#analyze_double_optimization(sa_tree_name, morpho_alignment_name)

si_optimized_tree = Tree(tree_dir + "geo_science_optimized_morpho_scaled_interval=[0,0.3].tree")
sa_optimized_tree = Tree(tree_dir + "geo_science_optimized_morpho_scaled_avg=0.03.tree")

compare_trees(si_optimized_tree, get_optimized_tree(si_tree_name, morpho_alignment_name), True)
compare_trees(sa_optimized_tree, get_optimized_tree(sa_tree_name, morpho_alignment_name), True)
   





    

0.0 --> 0.0 (0)
3e-09 --> 0.001115 (371665.667)
3e-09 --> 1e-06 (332.333)
3e-09 --> 1e-06 (332.333)
0.183912 --> 0.107757 (0.414)
3e-09 --> 1e-06 (332.333)
0.2073 --> 0.133791 (0.355)
3e-09 --> 1e-06 (332.333)
0.0397445 --> 0.035313 (0.111)
3e-09 --> 0.003736 (1245332.333)
0.0386434 --> 0.032649 (0.155)
0.0711548 --> 0.051063 (0.282)
0.3 --> 0.470862 (0.57)
3e-09 --> 1e-06 (332.333)
0.00599177 --> 0.005875 (0.019)
3e-09 --> 1e-06 (332.333)
0.135586 --> 0.074943 (0.447)
0.047618 --> 0.034433 (0.277)
0.288668 --> 0.159099 (0.449)
3e-09 --> 0.011576 (3858665.667)
0.230283 --> 0.177103 (0.231)
0.3 --> 0.189462 (0.368)
0.0468625 --> 0.028962 (0.382)
0.199805 --> 0.159596 (0.201)
0.0526988 --> 0.026045 (0.506)
0.010724 --> 0.017057 (0.591)
3e-09 --> 0.005185 (1728332.333)
0.3 --> 0.009339 (0.969)
0.00641209 --> 0.01317 (1.054)
3e-09 --> 1e-06 (332.333)
3e-09 --> 1e-06 (332.333)
3e-09 --> 1e-06 (332.333)
0.0300214 --> 0.026443 (0.119)
0.00613446 --> 1e-06 (1.0)
3e-09 --> 0.001352 (450665.667)